In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# reading all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [3]:
len(words)

29681

In [4]:
# building the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# building the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% of the data
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% of the data
Xte, Yte = build_dataset(words[n2:]) # 10% of the data

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [21]:
# MLP revisited
n_embd = 10 # the dimensionality of the embeddings
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd),          generator=g) # the embedding matrix
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * 0.2 # the first layer of the MLP
b1 = torch.randn((n_hidden),                    generator=g) * 0.01 # the bias of the first layer
W2 = torch.randn((n_hidden, vocab_size),        generator=g) * 0.01 # the second layer of the MLP
b2 = torch.randn((vocab_size),                  generator=g) * 0 # the bias of the second layer

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True


11897


In [22]:
# same optimization as in part 2
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construction
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  hpreact = embcat @ W1 + b1 # hidden layer pre-activation
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # tracking the stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: loss = {loss.item():.4f}')
  lossi.append(loss.log10().item())

  break

      0/ 200000: loss = 3.3134


In [8]:
# plt.figure(figsize=(20,10))
# plt.imshow(h.abs() > 0.99, cmap='gray', interpolation='nearest')

In [23]:
h

tensor([[ 0.6043, -0.0479,  0.8024,  ..., -0.4078,  0.5999,  0.9707],
        [-0.6281, -0.7772, -0.9030,  ...,  0.2717, -0.6118, -0.7203],
        [ 0.6310, -0.2498,  0.8875,  ..., -0.6563,  0.6171,  0.9729],
        ...,
        [ 0.5198,  0.5328,  0.8333,  ...,  0.3514, -0.5817,  0.0260],
        [-0.0305, -0.6984, -0.5057,  ..., -0.6223, -0.6202, -0.1715],
        [-0.4985, -0.8770, -0.9670,  ...,  0.0131,  0.8928, -0.4755]],
       grad_fn=<TanhBackward0>)

In [24]:
h.shape

torch.Size([32, 200])

In [11]:
# plt.hist(h.view(-1).tolist(), 50);

In [12]:
# plt.hist(hpreact.view(-1).tolist(), 50);

In [25]:
logits[0] # logits should be much much closer to zero

tensor([ 0.0888,  0.0006, -0.0109, -0.0595, -0.1520, -0.1342,  0.0086,  0.0184,
         0.0111,  0.0159, -0.0784,  0.0304, -0.0553, -0.0540,  0.0910,  0.0342,
        -0.1650,  0.1057,  0.0964, -0.1391, -0.0506,  0.0889, -0.0902, -0.0723,
        -0.0808,  0.0195,  0.0012], grad_fn=<SelectBackward0>)

In [26]:
# probability for any character
-torch.tensor(1/27.0).log()

tensor(3.2958)

In [27]:
# 4 dimensional example of the issue
logits = torch.tensor([1.0, 1.0, 1.0, 1.0])
# logits = torch.randn(4) * 10
probs = torch.softmax(logits, dim=0)
loss = -probs[2].log()
logits, probs, loss

(tensor([1., 1., 1., 1.]),
 tensor([0.2500, 0.2500, 0.2500, 0.2500]),
 tensor(1.3863))

In [16]:
# plt.plot(lossi)

In [28]:
@torch.no_grad() # this ensures that we don't save gradients
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split]
  emb = C[x] # (N, block_size, n_embd) embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors (N, block_size * n_embd)
  h = torch.tanh(embcat @ W1 + b1) # hidden layer (N, n_hidden)
  logits = h @ W2 + b2 # output layer (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())
  
split_loss('train')
split_loss('val')
split_loss('test')

train 3.2308545112609863
val 3.2318897247314453
test 3.230901002883911


In [18]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10) # for reproducibility

for _ in range(20):

  out = []
  context = [0] * block_size # initialize with all dots
  while True:
    # forward pass
    emb = C[torch.tensor([context])] # embedding
    h = torch.tanh(emb.view(1, -1) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    probs = F.softmax(logits, dim=1) # probabilities
    
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    
    # shift the context window and track the sample
    context = context[1:] + [ix]
    out.append(ix)
    
    # stop if we reach the end
    if ix == 0:
      break

  print(''.join(itos[i] for i in out)) # decode and print the sample

erbuagmyazzszevvndhfyaloqetursjejgrlrg.
azcjedgeliihsmpqppaledeixvqoaxarveptzkmhvkrpcn.
szdbvrgxhgmiest.
jcjrwjkjqqxntpucfkubzkgvdtrxyqqufzehgykmyjqvskkyaazhjl.
cakyybsufpzbjhlvjuren.
crtvwsvevoecmpfvdxrfqyxoeobhggptixjsfcueywh.
hmreolvii.
qiif.
cvpkxwvllayodtabfsztjnegoikvsblitgfkxvbssdvsu.
oeewrkepacmvyyxavttudkjgczekhgkgtipvtudt.
iwbddihzvorlq.
cjakepaugniagcrpppfmiipzjgvonxet.
jydnblksyzporygvogsvmwosfzpbskknymjcxtkncjbmu.
jtew.
aoa.
copmrlxyzdcqbcep.
ccafivsnicimhtewmosagouwzlrurdckappomtcokwxtnbnzkdphfam.
kconfffuecxkhosy.
sa.
ascnpc.
